# Costruire con i modelli Mistral

## Introduzione

Questa lezione tratterà:
- Esplorare i diversi modelli Mistral
- Comprendere i casi d’uso e gli scenari per ogni modello
- Esempi di codice mostrano le caratteristiche uniche di ciascun modello.


## I modelli Mistral

In questa lezione esploreremo 3 diversi modelli Mistral: **Mistral Large**, **Mistral Small** e **Mistral Nemo**.

Ognuno di questi modelli è disponibile gratuitamente sul marketplace dei modelli di Github. Il codice in questo notebook utilizzerà questi modelli per eseguire il codice. Qui trovi maggiori dettagli su come utilizzare i modelli di Github per [prototipare con modelli di intelligenza artificiale](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 è attualmente il modello di punta di Mistral ed è pensato per l’uso aziendale.

Il modello rappresenta un aggiornamento rispetto al Mistral Large originale offrendo:
-  Finestra di contesto più ampia - 128k contro 32k
-  Prestazioni migliori in compiti di matematica e programmazione - 76,9% di accuratezza media contro 60,4%
-  Maggiore capacità multilingue - le lingue supportate includono: inglese, francese, tedesco, spagnolo, italiano, portoghese, olandese, russo, cinese, giapponese, coreano, arabo e hindi.

Grazie a queste caratteristiche, Mistral Large eccelle in:
- *Retrieval Augmented Generation (RAG)* - grazie alla finestra di contesto più ampia
- *Function Calling* - questo modello supporta nativamente le chiamate di funzione, permettendo l’integrazione con strumenti esterni e API. Le chiamate possono essere effettuate sia in parallelo che in sequenza.
- *Code Generation* - il modello è particolarmente efficace nella generazione di codice Python, Java, TypeScript e C++.


In questo esempio, utilizziamo Mistral Large 2 per eseguire un pattern RAG su un documento di testo. La domanda è scritta in coreano e chiede delle attività dell'autore prima dell'università.

Si utilizza il modello Cohere Embeddings per creare gli embedding sia del documento di testo che della domanda. In questo esempio, viene utilizzato il pacchetto Python faiss come archivio vettoriale.

Il prompt inviato al modello Mistral include sia la domanda che i frammenti recuperati che sono simili alla domanda. Il modello fornisce poi una risposta in linguaggio naturale.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small è un altro modello della famiglia Mistral, appartenente alla categoria premier/enterprise. Come suggerisce il nome, questo modello è uno Small Language Model (SLM). I vantaggi nell’utilizzare Mistral Small sono:
- Risparmio sui costi rispetto agli LLM Mistral come Mistral Large e NeMo - riduzione del prezzo dell’80%
- Bassa latenza - risposte più rapide rispetto agli LLM di Mistral
- Flessibilità - può essere distribuito in diversi ambienti con meno restrizioni sulle risorse necessarie.

Mistral Small è ideale per:
- Attività basate su testo come riassunti, analisi del sentiment e traduzioni.
- Applicazioni che richiedono richieste frequenti grazie alla sua convenienza economica
- Attività di codice a bassa latenza come revisione e suggerimenti di codice


## Confronto tra Mistral Small e Mistral Large

Per mostrare le differenze di latenza tra Mistral Small e Large, esegui le celle qui sotto.

Dovresti notare una differenza nei tempi di risposta tra 3 e 5 secondi. Nota anche la lunghezza e lo stile delle risposte con lo stesso prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Rispetto agli altri due modelli trattati in questa lezione, Mistral NeMo è l’unico modello gratuito con licenza Apache2.

Viene considerato un’evoluzione rispetto al precedente LLM open source di Mistral, Mistral 7B.

Altre caratteristiche del modello NeMo sono:

- *Tokenizzazione più efficiente:* Questo modello utilizza il tokenizer Tekken invece del più comune tiktoken. Questo garantisce prestazioni migliori su più lingue e sul codice.

- *Finetuning:* Il modello base è disponibile per il finetuning. Questo offre maggiore flessibilità per i casi d’uso in cui il finetuning può essere necessario.

- *Function Calling nativo* - Come Mistral Large, anche questo modello è stato addestrato sul function calling. Questo lo rende unico, essendo uno dei primi modelli open source a offrire questa funzionalità.


## Mistral NeMo

Rispetto agli altri due modelli trattati in questa lezione, Mistral NeMo è l’unico modello gratuito con licenza Apache2.

È considerato un miglioramento rispetto al precedente LLM open source di Mistral, Mistral 7B.

Altre caratteristiche del modello NeMo sono:

- *Tokenizzazione più efficiente:* Questo modello utilizza il tokenizer Tekken invece del più comune tiktoken. Questo garantisce prestazioni migliori su più lingue e sul codice.

- *Finetuning:* Il modello base è disponibile per il finetuning. Questo offre maggiore flessibilità per i casi d’uso in cui il finetuning può essere necessario.

- *Function Calling nativo* - Come Mistral Large, anche questo modello è stato addestrato per il function calling. Questo lo rende unico, essendo uno dei primi modelli open source a offrire questa funzionalità.


### Confronto tra Tokenizer

In questo esempio, vedremo come Mistral NeMo gestisce la tokenizzazione rispetto a Mistral Large.

Entrambi gli esempi utilizzano lo stesso prompt, ma dovresti notare che NeMo restituisce meno token rispetto a Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## L'apprendimento non si ferma qui, continua il tuo percorso

Dopo aver completato questa lezione, dai un'occhiata alla nostra [collezione di apprendimento sull'IA generativa](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) per continuare a migliorare le tue conoscenze sull'IA generativa!



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni di carattere critico, si raccomanda una traduzione professionale effettuata da un essere umano. Non siamo responsabili per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
